In [ ]:
from math import exp
from kubernetes import client, config

def scaling_rate_count(workload_perf, resource_util):
    # calculate the scaling rate based on current workload performance and resource utilization
    # To be updated.......
    scaling_rate = exp(workload_perf + resource_util)
    return scaling_rate

def set_node_affinity(deployment):
    # Define the node affinity rule
    affinity = client.V1Affinity(
        node_affinity=client.V1NodeAffinity(
            required_during_scheduling_ignored_during_execution=client.V1NodeSelector(
                node_selector_terms=[
                    client.V1NodeSelectorTerm(
                        match_expressions=[
                            client.V1NodeSelectorRequirement(
                                key="kubernetes.io/hostname",
                                operator="In",
                                values=["microk8snode1"]  # scaled pods will be scheduled on the node; Multiple nodes can be added here
                            )
                        ]
                    )
                ]
            )
        )
    )
    
    # Set the affinity in the deployment's pod template
    if deployment.spec.template.spec.affinity:
        deployment.spec.template.spec.affinity.node_affinity = affinity.node_affinity
    else:
        deployment.spec.template.spec.affinity = affinity

def scale_deployment_replicas(deployment_name, namespace, delta_replica_count=2, scaling_rate=1):
    """
    Scale the replica count of a deployment.
    """
    # Load the kube config
    config.load_kube_config()
    # Create an AppsV1Api instance
    api = client.AppsV1Api()

    # Fetch the existing deployment
    deployment = api.read_namespaced_deployment(deployment_name, namespace)
    
    # Update the replica count
    deployment.spec.replicas = deployment.spec.replicas + delta_replica_count * scaling_rate

    # Set node affinity rules to make sure the pod runs on a specific node
    # This can be set when rquire the node affinity. This can be cancelled when not required.
    set_node_affinity(deployment) 
    
    # Apply the updated deployment spec
    api.patch_namespaced_deployment(deployment_name, namespace, deployment)
    
    #print how many new pods has been added successfully
    print("Deployment {} in namespace {} scaled to {} replicas".format(deployment_name, namespace, deployment.spec.replicas))




In [ ]:
# Test usage
deployment_name = "reviews-v1"
namespace = "default"
delat_replica_count = 1  # adjust this to your desired count
scale_deployment_replicas(deployment_name, namespace, delta_replica_count = delat_replica_count, scaling_rate=1)